In [ ]:
from pyfasta import Fasta
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
import random
import use_CNN



from sklearn.utils.class_weight import compute_sample_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix,mean_squared_error,matthews_corrcoef
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import SelectFromModel, f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

from keras.models import Sequential
from keras.layers import LSTM, Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras import metrics
from keras import backend
from keras.callbacks import ModelCheckpoint

from itertools import combinations

from scipy.optimize import minimize

In [ ]:
def pickle_load(path):
    import pickle
    with open(path, 'rb') as f:
        x_train = pickle.load(f)
        y_train = pickle.load(f)
        x_test = pickle.load(f)
        y_test = pickle.load(f)
    return x_train, y_train, x_test, y_test

In [ ]:
"""
PSO select CNN
"""

In [ ]:
## time range of data
## training set: 1968 - 2010
## test set: 2011 - 2016
path = "./data_1968-2010_2011-2016.pkl"
x_train, y_train, x_test, y_test = pickle_load(path)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


In [ ]:
"""
this part runs for a very long time
be prepared
"""
### ------------ PSO selection of optimal CNN ------------ ###
import numpy as np
import pandas as pd
import random
from datetime import datetime
import traceback
from keras import backend



### helper function:
## convert CNN vector between original form and [0,1]

# Number of layers of CNN 1 4
# Number of filters 32 256
# Kernel size  2 5
# Stride 1 3
# Pooling size 1 4
# Dropout probability 0.1 0.5
# Number of dense layers 1 3
# Number of neurons in dense layers 72 256
def convert_cnn_vector(x):
    ## number of dense layers could be smaller than 3
    new_x = np.zeros(len(x))
    ## convert normed to original vector
    if x[0]<=1:

        for i in range(0, 20, 5):
            if x[i]>=0:
                new_x[i] = 32 + np.round(x[i]*(256-32))
                new_x[i+1] = 2 + np.round(x[i+1]*(5-2)) if x[i+1]>=0 else 2
                new_x[i+2] = 1 + np.round(x[i+2]*(3-1)) if x[i+2]>=0 else 1
                new_x[i+3] = 1 + np.round(x[i+3]*(4-1)) if x[i+3]>=0 else 1
                new_x[i+4] = 0.1 + x[i+4]*(0.5-0.1) if x[i+4]>=0 else 0.1
            else:
                new_x[i] = 0
                new_x[i+1] = 0
                new_x[i+2] = 0
                new_x[i+3] = 0
                new_x[i+4] = 0
        for i in range(20, len(x), 2):
            if x[i] >= 0:
                new_x[i] = 72 + np.round(x[i]*(256-72)) 
                new_x[i+1] = 0.1 + x[i+1]*(0.5-0.1) if x[i+1]>=0 else 0.1
            else:
                new_x[i] = 0
                new_x[i+1] = 0
    ## convert to normalized vector
    else:
        for i in range(0, 20, 5):
            new_x[i] = (x[i] - 32)/(256-32) if x[i] != 0 else -0.5
            new_x[i+1] = (x[i+1] - 2)/(5-2) if x[i+1] != 0 else -0.5
            new_x[i+2] = (x[i+2] - 1)/(3-1) if x[i+2] != 0 else -0.5
            new_x[i+3] = (x[i+3] - 1)/(4-1) if x[i+3] != 0 else -0.5
            new_x[i+4] = (x[i+4] - 0.1)/(0.5-0.1) if x[i+4] != 0 else -0.5
        for i in range(20, len(x), 2):
            new_x[i] = (x[i] - 72)/(256-72) if x[i] != 0 else -0.5
            new_x[i+1] = (x[i+1] - 0.1)/(0.5-0.1) if x[i+1] != 0 else -0.5
    # print("x0: ",new_x[0])
    ## return new and original
    return(new_x)
#### collection of objective functions ####
def obj(x, trainx, trainy, testx, testy, ratio=0.0):
    new_x = convert_cnn_vector(x)
    
    if not use_CNN.check_cnn_valid(new_x,trainx.shape[1:]):
        new_x = use_CNN.random_generate_cnn(None, 1, n_cnn, n_dense, p_cnn, p_dense, trainx.shape[1:], fix_dense=fixed)
        new_x = new_x.reshape(new_x.shape[1])
    x = convert_cnn_vector(new_x)
    ## get objective value
    model = use_CNN.make_cnn(new_x, trainx.shape[1:])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(trainx, trainy, epochs=10, batch_size=600)
    _, train_result = use_CNN.DL_mcc(model, trainx, trainy)

    #pred_score = model.predict(testx)
    _, test_result = use_CNN.DL_mcc(model, testx, testy)

    return(train_result['MCC']*ratio + (1-ratio)*test_result['MCC'])
def test_obj(x, trainx, trainy, testx, testy, ratio=0.4):
    new_x = convert_cnn_vector(x)
    if not use_CNN.check_cnn_valid(new_x,trainx.shape[1:]):
        new_x = use_CNN.random_generate_cnn(None, 1, n_cnn, n_dense, p_cnn, p_dense, trainx.shape[1:], fix_dense=fixed)
        new_x = new_x.reshape(new_x.shape[1])
    x = convert_cnn_vector(new_x)
    ## get objective value

    model = use_CNN.make_cnn(new_x, trainx.shape[1:])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    ## no training
    return(np.sum(new_x))

### generate a vector of length dimension, with 0~1
### n_output: number of output
def initial_position_int(dimension, n_output, shape):
    ini_pos = np.zeros((n_output, dimension))
    for i in range(ini_pos.shape[0]):
        x = use_CNN.random_generate_cnn(None, 1, n_cnn, n_dense, p_cnn, p_dense, shape, fix_dense=fixed)
        # print(x[0])
        norm_x = convert_cnn_vector(x[0])
        if i > 0:
            for j in range(0, i):
                while np.allclose(norm_x, ini_pos[j,:]):
                    x = use_CNN.random_generate_cnn(None, 1, n_cnn, n_dense, p_cnn, p_dense, shape, fix_dense=fixed)
                    norm_x = convert_cnn_vector(x[0])
        ini_pos[i,:] = norm_x
    return(ini_pos)

#### ---------------- neighbor function ------------------- ####   
def defign_von_neumann_neighbor(swarm):
    r = 0
    c = 0
    l = len(swarm)
    l_sqrt_int = int(np.floor(np.sqrt(l)))
    for i in range(l_sqrt_int, 0, -1):
        if l % i == 0:
            r = i
            c = int(l/r)
            break
    ## print("r ", r, "; c ", c)
    ## neighborhood mesh
    #print("-- mesh dim: ", r, ", ", c)
    mesh = np.zeros((r,c))
    check_mesh = []
    for i in range(l):
        col = int((i+1)%c - 1)
        row = int(np.ceil((i+1)/c) - 1)
        mesh[row][col] = i    ### use location to find idx of swarm
        check_mesh.append((row, col))   ### use idx to find location
    return(mesh, check_mesh)

def find_local_best_von_neumann(swarm, idx, mesh, check_mesh, length):
    row, col = check_mesh[idx]
    local_best = -1     ## local best
    locao_best_pos = []  ## local best location
    ### go through the neighbor of idx
    for i in range(row - length, row + length +1):
        for j in range(col - length, col + length + 1):
            if i < 0: 
                i += mesh.shape[0]
            elif i >= mesh.shape[0]:
                i = i - mesh.shape[0] 
            if j <0:
                j += mesh.shape[1]
            elif j >= mesh.shape[1]:
                j = j - mesh.shape[1]
            dist = np.absolute(row + col - i - j) 
            
            ## update local best
            if dist <= length:
                target = int(mesh[i][j])
                if local_best < swarm[target].best_value:
                    local_best = swarm[target].best_value
                    local_best_pos = swarm[target].pos_best.copy()
    return(local_best_pos, local_best)

### ------------------- Particle ------------------ ###
class Particle:
    def __init__(self,x0):
        self.position=[]          # particle position, binary vector
        self.velocity=[]          # particle velocity
        self.pos_best=[]          # best position individual
        self.best_value=-1          # best error individual
        self.value=-1               # error individual
        
        
        for i in range(0, len(x0)):
            self.velocity.append(random.uniform(-1,1))
            self.position.append(x0[i])

    # evaluate current fitness
    def evaluate(self, costFunc, xtrain, ytrain, xtest, ytest, tv_ratio):
        self.value = costFunc(self.position, xtrain, ytrain, xtest, ytest, tv_ratio)

        # check to see if the current position is an individual best
        if self.value > self.best_value or self.best_value==-1:
            self.pos_best=self.position
            self.best_value=self.value

    # update new particle velocity
    def update_velocity(self, best_global_pos):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=2        # cognative constant
        c2=2        # social constant
        # w1 = 0.9
        # w2 = 0.4
        # w = (w1-w2)*(maxiter - currentiter)/maxiter + w2
        for i in range(0, len(self.position)):
            r1 = random.random()
            r2 = random.random()

            vel_cognitive = c1*r1*(self.pos_best[i]-self.position[i])
            vel_social = c2*r2*(best_global_pos[i]-self.position[i])
            self.velocity[i] = w*self.velocity[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self, dimension):
        tmp_pos = np.array(self.position) + np.array(self.velocity)
        tmp_pos[tmp_pos>1] = 1
        tmp_pos[tmp_pos<0] = 0
        self.position=tmp_pos
        
#### ---------------- PSO main function ------------------- ####

def PSO_feature_selection(costFunc, dimension, xtrain, ytrain, xtest, ytest, tv_ratio, num_particles = 25, 
                          maxiter = 30, verbose = False, use_neighbor = True):
    length = 1        # neighborhood dist to be checked
    best_value = -1   # best global value/MCC
    best_pos = []     # best global position
    best_trend = []   # the trend of best value over iteration
    
    ## ---- initialize position of particles ---- ##
    
    ini_pos = initial_position_int(dimension, num_particles, xtrain.shape[1:])
    swarm = []
    for i in range(num_particles):
        swarm.append(Particle(ini_pos[i]))
        
    ## ---- index to find neighbor ---- ##
    mesh, check_mesh = defign_von_neumann_neighbor(swarm)
    
    ## ---- optimize ---- ##
    for i in range(maxiter):
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j].evaluate(costFunc, xtrain, ytrain, xtest, ytest, tv_ratio)

            # determine if current particle is the best (globally)
            if not use_neighbor:
                if swarm[j].value > best_global_value or best_global_value == -1:
                    best_pos = swarm[j].position.copy()
                    best_value = float(swarm[j].value)
                
        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            if use_neighbor:
                best_pos, best_value = find_local_best_von_neumann(swarm, j, mesh, check_mesh, length)
            swarm[j].update_velocity(best_pos)
            swarm[j].update_position(dimension)
            
        tmp_v = -1
        for j in range(0,num_particles):    
            if swarm[j].best_value > tmp_v:
                tmp_v = swarm[j].best_value
        best_trend.append(tmp_v)
        if i>=45 and len(best_trend)>2:
            if np.absolute(best_trend[-1] - best_trend[-2]) < 0.000001:
                break
        if verbose:
            print("-- PSO iteration: ", i)
            print("---- Global best value: ", tmp_v)
            if i % 10 ==0:
                util.send_email("Iteration report", str(i))
            # for j in range(num_particles):
            #     print("------ Swarm ", j, ", best: ", swarm[j].best_value, ", value: ", swarm[j].value)
        ## clear session for every iteration
        backend.clear_session()

    best_pos = []
    for i in range(len(swarm)):
        if swarm[i].best_value == best_trend[-1]:
            best_pos = swarm[i].pos_best.copy()
            break
    
    return(best_trend, best_pos)


def test_convert_cnn_vector():
    norm_x = [0.5, 0.2, -0.2, 0.4, 0.1, 0.5, 0.2, 0.3, 0.4, 0.1, 0.5, 0.2, 0.3, 0.4, 0.1, 0, 0, 0, 0, 0, 0.2, 0.4, 0.1, 0.3, 0.6,0.2]
    #x = [70, 4, 1, 3, 0.4, 210, 5, 2, 3, 0.45, 210, 5, 2, 2, 0.45, 0, 0, 0, 0, 3, 100, 0.5, 112, 0.2, 128, 0.3]
    norm_x = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.4, 0.1, 0.3, 0.6,0.2]
    
    print(norm_x)
    org_x = convert_cnn_vector(norm_x)
    print("org x: ", org_x)
    norm_x2 = convert_cnn_vector(org_x)
    print("norm x 2: ",norm_x2)
    org_x2 = convert_cnn_vector(norm_x2)
    print("org x2: ", org_x2)
    print(np.allclose(org_x, org_x2, rtol = 1e-4))
    print("Check valid: ",use_CNN.check_cnn_valid(norm_x, [100,116,1]))
def test_random_cnn_struc():
    struc = use_CNN.random_generate_cnn(1234567890, 1, 4, 3, 5, 2, [10,10,1], fix_dense=False)
    print(struc.shape)
    # print(struc[0][0:20])
    print(struc[0][0:20].reshape(4,5))
    print(struc[0][20:].reshape(3,2))
def test_make_cnn():
    x = np.array([70, 4, 1, 0.4, 3, 210, 5, 2, 0.45, 3, 210, 5, 2, 0.45, 3, 0, 0, 0, 0, 3, 100, 0.5, 112, 0.2])
    x = [0.5, 0.2, 0.3, 0.4, 0.1, 0.5, 0.2, 0.3, 0.4, 0.1, 0.5, 0.2, 0.3, 0.4, 0.1, 0, 0, 0, 0, 0, 0.2, 0.4, 0.1, 0.3]
    new_x = convert_cnn_vector(x)
    new_x = use_CNN.random_generate_cnn(None, 1, 4, 3, 5, 2, [100,10,1], fix_dense=False)
    new_x = new_x[0]
    print(new_x[0:20].reshape(4,5))
    model = use_CNN.make_cnn(new_x, [100,10,1])
    print(model.summary())
def declair_global_variable():
    global n_cnn, n_dense, p_cnn, p_dense, fixed, seed
    n_cnn, n_dense, p_cnn, p_dense, fixed, seed = 4, 3, 5, 2, True, 1489683273
    print("Len of vec: ", n_cnn*p_cnn+n_dense*p_dense)
def read_support(file):
    opt_pos = []
    with open(file, "r") as f:
        for line in f:
            line = line.rstrip()
            opt_pos = list(map(int, line.split("\t")))
    f.close()
    return(np.array(opt_pos, dtype=bool))

def main(x_train, y_train, x_test, y_test):

    ## training/ validation ratio
    tv_ratio = 0.0
    declair_global_variable()
    
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    best_trend, best_pos = PSO_feature_selection(obj, n_cnn*p_cnn+n_dense*p_dense, 
        x_train, y_train, x_test, y_test, tv_ratio, maxiter = 50, verbose = True)
    orig_best_pos = convert_cnn_vector(best_pos)


    best_trend_file = "./output/PSO_CNN_struc_best_trend_sort3d.txt"
    best_pos_file = "./output/PSO_CNN_struc_best_pos_sort3d.txt"
    
    FOUT = open(best_trend_file, "w")
    FOUT.write("\t".join(str(x) for x in best_trend))
    FOUT.close()
    
    FOUT = open(best_pos_file, "w")
    FOUT.write("\t".join(str(x) for x in orig_best_pos))
    FOUT.close()

main(x_train, y_train, x_test, y_test)








In [ ]:
## train a model with early stop
## 
## perc: positive case change by perc, negative case change by -perc
def cust_rebalance(y_train, train_weight, perc=0.1):
    p = np.bincount(y_train)
    weights, counts = np.unique(train_weight, return_counts=True)
    print("Initial weights:", weights)

    for i, item in enumerate(p):
        idx = counts == item
        if i == 0:
            trainw_idx = train_weight == weights[idx]
            train_weight[trainw_idx] = weights[idx]*(1-perc)
        elif i==1:
            trainw_idx = train_weight == weights[idx]
            train_weight[trainw_idx] = weights[idx]*(1+perc)
    print("Final weights:", np.unique(train_weight, return_counts=False))

    
def run_early_stop(struc_file, x_train, y_train, x_test, y_test, epoch = 50):
    
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    ## read CNN structure
    model = use_CNN.read_cnn_struc(struc_file, x_train.shape[1:], regression=False)

    # optimizer = optimizers.SGD(lr=0.01, decay=1e-8, momentum=1, nesterov=True)
    # model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    ## compute sample weight 
    train_weight = compute_sample_weight("balanced",y_train)
    valid_weight = compute_sample_weight("balanced",y_test)
    # rebalance(train_weight, valid_weight)

    cust_rebalance(y_train, train_weight, perc=0.4)
    ## checkpoint            
    filename="CNN_model_{epoch:02d}-{val_acc:.2f}.hdf5"
    filepath = os.path.join("./output/model",filename)

    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # fit
    history = model.fit(x_train, y_train, epochs=epoch, batch_size=600, sample_weight=train_weight, 
        callbacks=callbacks_list, validation_split=0.2)
    # Final evaluation of the model
    scores = model.evaluate(x_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    _, train_result = use_CNN.DL_mcc(model, x_train,y_train)
    _, test_result = use_CNN.DL_mcc(model, x_test,y_test)
    
    
struc_file = "./PSO_CNN_struc_best_pos_2018-3-22_artificial96.txt"
run_early_stop(struc_file, x_train, y_train, x_test, y_test, epoch = 200)
    

In [ ]:
"""
train CNN model for additional cases
"""
paths = ["./data_1968-1997_1998.pkl",
        "./data_1968-1997_1998.pkl",
        "./data_1968-1998_1999.pkl",
        "./data_1968-1999_2000.pkl",
        "./data_1968-2000_2001.pkl",
        "./data_1968-2001_2002.pkl",
        "./data_1968-2002_2003.pkl",
        "./data_1968-2003_2004.pkl",
        "./data_1968-2004_2005.pkl",
        "./data_1968-2005_2006.pkl",
        "./data_1968-2006_2007.pkl",
        "./data_1968-2007_2008.pkl",
        "./data_1968-2008_2009.pkl",
        "./data_1968-2009_2010.pkl",
        "./data_1968-2010_2011.pkl"]

x_train, y_train, x_test, y_test = pickle_load(paths[0])
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

run_early_stop(struc_file, x_train, y_train, x_test, y_test, epoch = 200)